<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>: Veton Abazovic<br>
<b>NetID</b>: va187<br>
<b>Recitation (01/02)</b>: 02<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:     VA


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

# Lab 7: Linear Models, Feature Engineering and Logistic Regression

** This lab is due Monday April 22, 2019 at 11:59pm (graded on accuracy and completeness) **

In this lab we will work through the process of:
1. implementing a linear model, defining loss functions, 
2. feature engineering,
3. minimizing loss functions using numeric libraries 
4. how to implement logistic regression model for a binary classification problem

We will use the toy tip calculation dataset from sns.

## Set up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

## Load the Tips Dataset

To begin with, we load the tips dataset from the `seaborn` library.  The tips data contains records of tips, total bill, and information about the person who paid the bill.

In [ ]:
data = sns.load_dataset("tips")
print("Number of Records:", len(data))
data.head()

# Part 1: Defining the Model and Feature Engineering

In lab 6, we defined a simple linear model with only one parameter. Now let's make a more complicated model that utilizes other features in a dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. 
### Task 1.1 Split the data into Y(tip) and X(all others)

In [ ]:
## BEGIN SOLUTION
tips = data['tip']
X = data.drop(columns = 'tip')
X
## END SOLUTION

## Task 1.2: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

solution: one-hot encoding! 

As discussed in lecture 9.2, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset.

In [ ]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input, data. 
    Hint: check pd.get_dummies
    """
    ### BEGIN SOLUTION
    return pd.get_dummies(data)
    ### END SOLUTION

In [ ]:
one_hot_X = one_hot_encode(X)
one_hot_X = one_hot_X[['size', 'total_bill', 'day_Thur', 'day_Fri', 'day_Sat', 'day_Sun', 'sex_Male', 'sex_Female', 'smoker_Yes', 'smoker_No', 'time_Lunch', 'time_Dinner']]
one_hot_X.head()

## Task 1.2: Defining the Model
Now that all of our data is numeric, let's define our model function. Note that X and thetas are matrices now. Use matrix products to compute the model.

In [ ]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    ### BEGIN SOLUTION
    return X.dot(thetas)
    ### END SOLUTION

In [ ]:
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

# Part 2: Fitting the Model: Numeric Methods
Recall in the previous lab we defined multiple loss functions and found optimal theta using the scipy.minimize function. Adapt the loss functions and optimization code from the previous lab (provided below) to work with your new linear model.

In [ ]:
from scipy.optimize import minimize

def abserror(y, y_hat):
    return np.abs(y-y_hat)

def sqerror(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    
    ## Notes on the following function call which you need to finish:
    # 
    # 0. the ... should be replaced with the average loss evaluated on 
    #       the data x, y using the model and appropriate loss function
    # 1. x0 are the initial values for THETA.  Yes, this is confusing
    #       but optimization people like x to be the thing they are 
    #       optimizing.
    # 2. We extract the 'x' entry in the dictionary which corresponds
    #       to the value of thetas at the optimum
    ### BEGIN SOLUTION
    return minimize(lambda theta: loss_function(model(theta, x), y).mean(), x0=np.random.rand(x.shape[1]))['x']
    ### END SOLUTION
    #return minimize(lambda theta: ..., x0=...)
minimize_average_loss(sqerror, linear_model, one_hot_X, tips)

# Part 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Task 3.1: Least Squares Solution
Recall that if we're fitting a linear model with the l2 loss function, we are performing least squares! Remember, we are solving the following optimization problem for least squares:

$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Write your answer in LaTeX in the cell below. Assume X is full column rank. You are given step 1 to get things started. You need to complete the steps 2-4 where step 4 shows the value of theta that optimizes given X and y.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ step 2: (X\theta - y)^T(X\theta -y)$$
$$(\theta^TX^T - y^T)(X\theta-y) $$
$$ \theta^T X^T X\theta - \theta^T X^T y -y^T X\theta +y^T y$$
$$ \theta^T X^T X\theta -2\theta^T X^T y + y^T y $$
$$ step 3 - Derivatives:$$
$$ d/theta(\theta^T X^T X\theta) - d/theta(2\theta^T X^T) + d/theta(y^T y) = 0$$
$$ (X^T X +( X^T X)^T)\theta - 2X^T y + 0 = 0$$
$$ 2X^T X\theta = 2X^Ty$$
### END SOLUTION

## Task 3.2: Solving for Theta
Using your formuala above, let us find the analytic solution for $\theta$. That is, the optimal numerical thetas for our tips dataset. Fill out the function below. Make sure you use the float type in your calculations using .astype(float) and use the np.linalg.inv function.

In [ ]:
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """
    ### BEGIN SOLUTION
    xTx = x.T.dot(x)
    xTy = x.T.dot(y)
    return np.linalg.inv(xTx).dot(xTy)
    ### END SOLUTION

In [ ]:
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
print("Our analytical loss is: ", sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean())
print("Our numerical loss is: ", sqerror(linear_model(minimize_average_loss(sqerror, linear_model, one_hot_X, tips), one_hot_X), tips).mean())

In [ ]:
assert np.isclose(sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean(), 59.733414754098362)

## Task 3.3: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

Answer: The reason our analytical loss is surprisingly much worse than our numerical loss is because when you use sum the numbers are close together and when you use the other method the numbers are further part. Using np.linalg.inv loses precision.

### BEGIN SOLUTION



### END SOLUTION

# Part 4 - Logistic Regression
In this part we will implement a logistic regression model with the tip data set. We will use Male and Female as the binary classification. We will use the LogisiticRegression model from sklearn to find a prediction model.

## task 4.1
** create the X (one-hot features) and y (sex) **

In [ ]:
X = one_hot_encode(X)
y = data['sex']
X = X[['total_bill','size','smoker_Yes','smoker_No','day_Thur','day_Fri','day_Sat', 'day_Sun','time_Lunch', 'time_Dinner']]
## END SOLUTION
print(X.head())
print(y.head())

## task 4.2 
Split the data set to training and test using sklearn train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
## BEGIN SOLUTION
X_train, X_test, y_train, y_test = train_test_split(X,y)
## END SOLUTION

print (X_train.head(), X_test.head(), y_train.head(), y_test.head())

## task 4.3 logistic regression model
Fit a logisitic regression model using LogisticRegression from sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
## BEGIN SOLUTION
logmodel = LogisticRegression().fit(X_train, y_train)
print(logmodel)
## END SOLUTION

## task 4.4 Predicting
Now we can use the logmodel from previous part to predict on the test data. Read documentation to see how. It should be just a one line of code.

In [ ]:
## BEGIN SOLUTION
predictions = logmodel.predict(X_test)
#predictions
## END SOLUTION

## task 4.5 reporting
use classification_report from sklearn.metrics to report the accuracy of the model.

In [ ]:
from sklearn.metrics import classification_report
## BEGIN SOLUTION
target_names = ['Female', 'Male']
print(classification_report(y_test, predictions, target_names=target_names))
## END SOLUTION

### Feedback
Please provide feedback on this lab.
* how would you rate this lab (from 1-10, 10-highest) : 9
* how can we improve his lab? :n/a

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_lab7.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; lab7 <br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Developed by A.D. Gunawardena. Credits: Josh Hug, Berkeley Data Science Lab